# Project 3:  Web Scraping
### Finding Underpriced RVs on Craigslist

![](https://snag.gy/WrdUMx.jpg)

In this project we will be practicing our web scraping skills.  You can use Scrapy or Python requests in order to complete this project.  It may be helpful to write some prototype code in this notebook to test your assumptions, then move it into a Python file that can be run from the command line.

> In order to run code from the command line, instead of the notebook, you just need to save your code to a file (with a .py extension), and run it using the Python interpreter:<br><br>
> `python my_file.py`

You will be building a process to scrape a single category of search results on Craigslist, that can easily be applied to other categories by changing the search terms.  The main goal is to be able to target and scrape a single page given a set of parameters.

**If you use Scrapy, provide your code in a folder.**

## Import your libraries for scrapy / requests / pandas / numpy / etc
Setup whichever libraries you need. Review past material for reference.

In [13]:
from scrapy.selector import Selector
from scrapy.http import HtmlResponse

import os
import numpy as np 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from collections import Counter

from sklearn.linear_model import LinearRegression, LassoCV, RidgeCV, ElasticNetCV
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler, Binarizer
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.metrics import r2_score

from statsmodels.stats.outliers_influence import variance_inflation_factor   

# Notebook Options:
from IPython.core.display import display, HTML
from IPython.core.interactiveshell import InteractiveShell
display(HTML("<style>.container { width:100% !important; }</style>"))
InteractiveShell.ast_node_interactivity = "all"

# Pandas Options: 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', 300)

# Point to Working Directory:
os.chdir('C://Users//Brian//Desktop//Project_3_github/project3')

### Summary:

I completely ran out of time for this one. I at least got the basics down if it comes up in the future. 



## 1.  Scrape for the largest US cities (non-exhaustive list)
Search, research, and scrape Wikipedia for a list of the largest US cities.  There are a few sources but find one that is in a nice table.  We don't want all cities, just signifficant cities.  Examine your source.  Look for what can be differentiable.

- Use requests
- Build XPath query(ies)
- Extract to a list
- Clean your list

## 1.2 Only retain cities with properly formed ASCII

Optionally, filter out any cities with impropper ASCII characters.  A smaller list will be easier to look at.  However you may not need to filter these if you spend more time scraping a more concise city list.  This list should help you narrow down the list of regional Craigslist sites.

## 2.  Write a function to capture current pricing information via Craigslist in one city.
Choose a city from your scraped data, then go to the cooresponding city section on Craigslist, searching for "rv" in the auto section.  Write a method that pulls out the prices.

In [ ]:
import scrapy
from scrapy.loader import ItemLoader
from craigslist.items import CraigslistItem



class CraigslistSpider(scrapy.Spider):
    name = 'brian'
    allowed_domains = ["craigslist.org"]
    start_urls = [
       "https://sfbay.craigslist.org/search/sfc/sss?query=rv&sort=rel&min_price=1000"
    ]

    def parse(self, response):
        title = response.xpath('//p[@class="result-info"]/a/text()').extract()
        url = response.xpath('//p[@class="result-info"]/a/@href').extract()
        price = response.xpath('//p[@class="result-info"]/span[@class="result-meta"]/span[@class="result-price"]/text()').extract()


        for title, url, price in zip(title, url, price):
            item = CraigslistItem()
            item['title'] = title
            item['url'] = url
            item['price'] = price
            yield item

In [15]:
df = pd.read_csv("base_spider.csv")
df.head()

,url,price,title
0,/sfc/rvd/6189570326.html,$29000,2016 Forest River RV Sabre Lite 28RL 5th Wheel - Rear Living Fifth Whe
1,/sfc/rvd/6188644100.html,$29000,2016 Forest River RV Sabre Lite 28RL 5th Wheel - Rear Living Fifth Whe
2,/sfc/rvs/6189877978.html,$9995,1991 Ford Tioga RV Motorhome
3,/sfc/rvs/6189876739.html,$9995,1991 Ford Tioga RV Motorhome
4,/sfc/rvd/6187174695.html,$28900,2017 Forest River RV Salem Hemisphere 282RK Travel Trailer



## 2.1 Create a mapping of cities to cooresponding regional Craigslist URLs

Major US cities on Craigslist typically have their own cooresponding section (ie: SFBay Area, NYC, Boston, Miami, Seattle, etc).  Later, you will use these to query search results for various metropolitian regions listed on Craigslist.  Between the major metropolitan Craigslist sites, the only thing that will differ is the URL's that correspond to them.

The point of the "mapping":  Create a data structure that allows you to iterate with both the name of the city from Wikipedia, with the cooresponding variable that that will allow you to construct each craigslist URL for each region.

> For San Francsico (the Bay Area metropolitan area), the url for the RV search result is:
> http://sfbay.craigslist.org/search/sss?query=rv
>
> The convention is http://[region].craigslist.org/search/sss?query=rf
> Replacing [region] with the cooresponding city name will allow you to quickly iterate through each regional Craigslist site, and scrape the prices from the search results.  Keep this in mind while you build this "mapping".


In [ ]:
dic = {'SFBay Area': 'https://sfbay.craigslist.org/search/sfc/sss?query=rv&sort=rel&min_price=1000', 
       'NYC' : 'https://newyork.craigslist.org/search/sfc/sss?query=rv&sort=rel&min_price=1000', 
       'Boston' : 'https://boston.craigslist.org/search/sfc/sss?query=rv&sort=rel&min_price=1000', 
       'Miami' : 'https://miami.craigslist.org/search/sfc/sss?query=rv&sort=rel&min_price=1000', 
       'Seattle' : 'https://seattle.craigslist.org/search/sfc/sss?query=rv&sort=rel&min_price=1000'}


## 3. Define a function to caculate mean and median price per city.

Now that you've created a list of cities you want to scrape, adapt your solution for grabbing data in one region site, to grab data for all regional sites that you collected, then calculate the mean and median price of RV results from each city.

> Look at the URLs from a few different regions (ie: portland, phoenix, sfbay), and find what they have in common.  Determine the area in the URL string that needs to change the least, and figure out how to replace only that portion of the URL in order to iterate through each city.

In [55]:
def rv_price(frame):
    
    p = pd.to_numeric(p)
       
    #p = []
    #for i in range(len(frame)):
    #    p.append(frame['price'][i][1:])
        
    return np.mean(p), np.median(p)

In [57]:
# Crawl Spider

In [ ]:
import scrapy
from craigslist.items import CraigslistItem
from scrapy.spiders import CrawlSpider, Rule
from scrapy.linkextractors import LinkExtractor
from bs4 import BeautifulSoup as bs


class CLSpider(CrawlSpider):

    name = 'clCrawl'
    allowed_domains = ["craigslist.org"]
    start_urls = ["https://sfbay.craigslist.org/search/sfc/sss?query=rv&sort=rel&min_price=1000"]

    rules=(Rule(LinkExtractor(restrict_xpaths = ['//a[@class="button next"]']),follow=True,callback='parse_d'),)


    def parse_d(self, response):
        title = response.xpath('//p[@class="result-info"]/a/text()').extract()
        url = response.xpath('//p[@class="result-info"]/a/@href').extract()
        price = response.xpath('//p[@class="result-info"]/span[@class="result-meta"]/span[@class="result-price"]/text()').extract()


        for title, url, price in zip(title, url, price):
            item = CraigslistItem()
            item['title'] = title
            item['url'] = url
            item['price'] = price
            yield item


## 4. Run your scraping process, and save your results to a CSV file.

In [58]:
dfc = pd.read_csv("cl_crawl_spider.csv")
dfc.head()

,url,price,title
0,/sfc/ctd/6157819665.html,$6995,Dodge Ram B3500 Extended Cargo Van Utility Bins Shelves 1-Ton 1 Owner 74K MILES
1,/sfc/rvs/6190304103.html,$28600,Your Whole Family Can Enjoy The 1999 Wilderness Northwest
2,/sfc/rvs/6190293010.html,$10500,2005 camper Outfitter Apex 8
3,/sfc/rvs/6190003466.html,$2500,River Flagstaff Microlite
4,/sfc/rvs/6189906858.html,$125000,Beautiful 2012 Fleetwood Bounder Classic - 35 K



## 5. Do an analysis of the RV market.

Go head we'll wait.  Anything notable about the data?


### 5.1 Does it makes sense to buy RVs in one region and sell them in another?

Assuming the cost of shipping or driving from one regional market to another.

### 5.2 Can you pull out the "make" from the markup and include that in your analyis?
How reliable is this data and does it make sense?

### 5.3 Are there any other variables you could pull out of the markup to help describe your dataset?


## 6. Move your project into scrapy (if you haven't used Scrapy yet)

>Start a project by using the command `scrapy startproject [projectname]`
> - Update your settings.py (review our past example)
> - Update your items.py
> - Create a spiders file in your `[project_name]/[project_name]/spiders` directory

You can update your spider class with the complete list of craigslist "start urls" to effectively scrape all of the regions.  Start with one to test.

Updating your parse method with the method you chose should require minimal changes.  It will require you to update your parse method to use the response parameter, and an item model (defined in items.py).


## 7.  Chose another area of Craigslist to scrape.

**Choose an area having more than a single page of results, then scrape multiple regions, multiple pages of search results and or details pages.**

This is the true exercise of being able to understand how to succesffuly plan, develop, and employ a broader scraping strategy.  Even though this seems like a challenging task, a few tweeks of your current code can make this very managable if you've pieced together all the touch points.  If you are still confused as to some of the milestones within this process, this is an excellent opportunity to round out your understanding, or help you build a list of questions to fill in your gaps.

_Use Scrapy!  Provide your code in this project directory when you submit this project._